This example shows a basic example of "Feff fitting" with larch for Cu metal at room temperature.

We start with some import statements, then do a "normal" first-shell fit
   1. read in data for Cu metal, Cu K edge
   2. do background subtraction and plot $\chi(k)$
   3. set the Fourier transform ranges
   4. define a Group of Parameters for the fit
   5. read in 1 Feff Path for Cu-Cu scattering
   6. run the fit and view the results
   
Then we will expand that to include 2 more paths, and show one way to constrain the distances parameters

In [1]:
# step 0: imports
from pathlib import Path
from larch.xafs import (pre_edge, autobk, xftf, xftr,
                        ff2chi, feffpath, feffit_transform, 
                        feffit_dataset, feffit, feffit_report)
from larch.fitting import param, param_group
from larch.io import read_ascii
from larch.plot.plotly_xafsplots import (plot_mu,
                                         plot_chik,
                                         plot_chir,
                                         plot_chifit,
                                         plot_chiq)

In [2]:
# step 1: read in the data and make mu(E) for room Temperature Cu

cu_data  = read_ascii(Path('Cu_foil_XAS_spectrum_Japan_database.txt'), 
                      labels='energy mu'
                      )
pre_edge(cu_data)
fig = plot_mu(cu_data, show_norm=True, show_e0=True)


In [3]:
# step 2: do and check the background subtraction
autobk(cu_data, rbkg=1.0, kweight=2)
fig = plot_chik(cu_data, kweight=2, show_window=True)

# First-shell fitting

In [4]:
# step 3: Fourier transform ranges ranges
xftf(cu_data, kmin=3, kmax=15, kweight=2, dk=4, window='kaiser')
xftr(cu_data, rmin=1.2, rmax=3, window='hanning', dr=0)

# Plot EXAFS in R-space
plot_chir(cu_data, show_window=True, rmax=7)


# Plot EXAFS in q-space
fig_chiq = plot_chiq(cu_data)
fig_chiq.add_plot(cu_data.k, 
             cu_data.chi*cu_data.k**2, 
             label='data in k-space')
fig_chiq.show()

# we can then definte Fourier tranform and fit ranges (first shell only: out to 3.0 Ang)
trans = feffit_transform(kmin=3, kmax=15, kweight=2, dk=4, window='kaiser', rmin=1.2, rmax=3.0)


In [5]:
# step 4: setup fitting Parameters: we will vary 
#   amplitude, e0, sigma2, and delta_R, and a third cumulant C3

pars = param_group(amp=param(1, vary=True),
                   de0=param(3, vary=True),
                   sig2=param(.005, vary=True),
                   delr=param(0.01, vary=True) )


In [6]:
# step 5: define a Feff Path for Cu-Cu scattering

# using a Path file from a previous Feff calculation for Zn-Se (shown elsewhere)
# here we give math expressions using the Parameters named in the Parameter group 
# to set the values for the Path Parameters in the EXAFS equation
path1 = feffpath(Path('FEFF_Cu/feff0001.dat'), 
                 label='Cu R=2.56Ang',
                 s02='amp', 
                 e0='de0', 
                 sigma2='sig2', 
                 deltar='delr', 
                 )

tmpsum = ff2chi([path1], params=pars)
fig = plot_chik(tmpsum, kweight=2, label='initial Feff Model', show=False)
fig.add_plot(cu_data.k, 
             cu_data.chi*cu_data.k**2, 
             label='data')

fig.show()

In [7]:
# step 6: run and view the fit

# now that we have data, a set of paths, and Fourier transform values, we can
# set up a "Feffit Dataset" and then fit that with the group of Parameters

# define dataset to include data, pathlist, transform
dset = feffit_dataset(data=cu_data, pathlist=[path1], transform=trans)

# do the fit...
result = feffit(pars, dset)

# print out the fit report
print(feffit_report(result))

=================== FEFFIT RESULTS ====================
[[Statistics]]
  n_function_calls     = 26
  n_variables          = 4
  n_data_points        = 116
  n_independent        = 14.7509871
  chi_square           = 64.4569422
  reduced chi_square   = 5.99544411
  r-factor             = 0.00196801
  Akaike info crit     = 29.7530953
  Bayesian info crit   = 32.5183353
 
[[Parameters]]
  amp                  =  0.9363936 +/- 0.2784151  (init= 1.0000000)
  de0                  =  5.8438438 +/- 3.4667688  (init= 3.0000000)
  delr                 =  0.0138026 +/- 0.0188216  (init= 0.0100000)
  sig2                 =  0.0092091 +/- 0.0023312  (init= 0.0050000)
 
[[Correlations]] (unreported correlations are <  0.100)
  amp, sig2            =  0.933
  de0, delr            =  0.921
  delr, sig2           =  0.170
  amp, delr            =  0.154
 
[[Dataset]]
  unique_id            = 'd3zplhok'
  fit space            = 'r'
  r-range              = 1.200, 3.000
  k-range              = 3.000, 1

In [8]:
# finally, plot the results
fig6, fig7 = plot_chifit(result.datasets[0])

# Add two more scattering path for the fitting (future)

Now we can add two more Paths for Cu scattering, and extend the R-range of the fit range to 3.5 Ang

In [9]:
# we will just change the Rmax to 3.5:
trans.rmax  = 3.5

# and we will update the parameters to 
# include a separate sigma2 for each path, 
# but we'll change 'delr' to 'alpha' and use this as fractional change in distance for each path.
pars = param_group(amp=param(1,vary=True),
             de0  = param(3,    vary=True),
             sig2 = param(.002, vary=True),
             sig2_p2 = param(.002, vary=True),
             sig2_p3 = param(.002, vary=True),
             alpha= param(0.001, vary=True) )


# and we will define 3 paths to use use these:
path1.deltar = 'alpha*reff'

path2 = feffpath('FEFF_Cu/feff0002.dat',  s02='amp', e0='de0',
                 sigma2='sig2_p2',  deltar='alpha*reff')

path3 = feffpath('FEFF_Cu/feff0003.dat', s02='amp', e0='de0',
                  sigma2='sig2_p3', deltar='alpha*reff')

# note that all 3 expressions for 'deltar' are the same 'alpha*reff'.  
# The value of 'reff' will be nominal Path distance

In [10]:
# now we can run that fit and view the results

dset  = feffit_dataset(data=cu_data, pathlist=[path1, path2, path3], transform=trans)

result = feffit(pars, dset)

print(feffit_report(result))

=================== FEFFIT RESULTS ====================
[[Statistics]]
  n_function_calls     = 43
  n_variables          = 6
  n_data_points        = 150
  n_independent        = 18.5707057
  chi_square           = 70.7543879
  reduced chi_square   = 5.62851358
  r-factor             = 0.00217122
  Akaike info crit     = 36.8407178
  Bayesian info crit   = 42.3702301
 
[[Parameters]]
  alpha                =  0.0054389 +/- 0.0074715  (init= 1.0000e-3)
  amp                  =  0.9303428 +/- 0.2781379  (init= 1.0000000)
  de0                  =  5.8674507 +/- 3.4708117  (init= 3.0000000)
  sig2                 =  0.0091704 +/- 0.0023326  (init= 0.0020000)
  sig2_p2              =  0.0130778 +/- 0.0080067  (init= 0.0020000)
  sig2_p3              =  0.0100181 +/- 0.0284746  (init= 0.0020000)
 
[[Correlations]] (unreported correlations are <  0.100)
  amp, sig2            =  0.933
  alpha, de0           =  0.925
  amp, sig2_p2         =  0.298
  sig2, sig2_p2        =  0.281
  amp, sig2_

In [11]:
# and plot the results
fig = plot_chifit(result.datasets[0], show_real=True)

In [12]:
from larch.plot.plotly_xafsplots import plot_paths_k, plot_paths_r

f = plot_paths_k(result.datasets[0], offset=-0.25)

In [13]:
f = plot_paths_r(result.datasets[0], offset=-0.25)